In [75]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [32]:
#code by pythonjar from stackoverflow, not me. Goal is to remove notifications on chrome
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

#start the webdriver
driver = webdriver.Chrome(chrome_options=chrome_options)

<ipython-input-32-dadd28cf965d>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [33]:
url = "https://www.facebook.com" #the website we want to scrape

driver.get(url) #enter the website

# Login

In [34]:
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

#enter username and password
username.clear()
username.send_keys("username")
password.clear()
password.send_keys("password")

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

# Enter the marketplace and search cars with prices between 100 and 300 mills

In [103]:
driver.get("https://www.facebook.com/marketplace/algiers/search?minPrice=1000000&maxPrice=3000000&sortBy=price_descend&query=voiture&exact=false")


# Get the items in a list

In [109]:
#scroll down the page until no more elements left or until 100 scrolls
for i in range(100):
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1.5)
    except:
         pass

#add the items links into the list
links=[]
elements = driver.find_elements_by_xpath('//a[contains(@class, "oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 p8dawk7l")]')
for ele in elements:

     links.append(ele.get_attribute('href'))


In [110]:
len(links) #number of items

72

# Get the description of each item

In [112]:
time.sleep(5)

item_desc = []
item_title= []
item_price= []
item_cat=[]
item_loc=[]


for l in links:
    driver.get(l)
    time.sleep(6)
    
    #clicks the "view more" button when there is one
    try:
        plus_button = driver.find_element_by_xpath('//span[contains(@class,"d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh jq4qci2q a3bd9o3v lrazzd5p oo9gr5id")]')
        plus_button.click()
    except:
        pass
    
    #get the location  
    try:
        loc = driver.find_element_by_xpath('//div[contains(@class,"sjgh65i0")]//span[contains(@class,"d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh e9vueds3 j5wam9gi knj5qynh m9osqain")]')
        loc_text= loc.text
    except: 
        loc_text= "No Location"
    item_loc.append(loc_text)  
    
    #get the category  
    try:
        cat = driver.find_element_by_xpath('//div[contains(@class,"aov4n071")]//a[contains(@class,"oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl m9osqain gpro0wi8 lrazzd5p")]')
        cat_text= cat.text
    except: 
        cat_text= "No Category"
    item_cat.append(cat_text)    

    #get the price
    try:
        price = driver.find_element_by_xpath('//div[contains(@class,"aov4n071")]//span[contains(@class,"d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j keod5gw0 nxhoafnm aigsh9s9 d3f4x2em fe6kdd0r mau55g9w c8b282yb mdeji52x a5q79mjw g1cxx5fr lrazzd5p oo9gr5id")]')
        price_text= price.text
    except: 
        price_text= "No Price"
    item_price.append(price_text)    

    #get the title                                         
    try:
        title= driver.find_element_by_xpath('//div[contains(@class,"j83agx80 cbu4d94t buofh1pr l9j0dhe7")]//span[contains(@class,"d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j keod5gw0 nxhoafnm aigsh9s9 qg6bub1s fe6kdd0r mau55g9w c8b282yb iv3no6db o0t2es00 f530mmz5 hnhda86s oo9gr5id")]')
        title_text = title.text
    except: 
        title_text="Title Error"
    item_title.append(title_text)    
    
      
    #get the description                                                                           
    try:
        #get the path to the item description element 
        single_item_desc = driver.find_element_by_xpath('//div[contains(@class,"ii04i59q a8nywdso f10w8fjw rz4wbd8a pybr56ya")]/div/span[contains(@class, "d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j keod5gw0 nxhoafnm aigsh9s9 d3f4x2em fe6kdd0r mau55g9w c8b282yb iv3no6db jq4qci2q a3bd9o3v knj5qynh oo9gr5id")]')
        
        #clean the data of some repeated sentences
        item_text = single_item_desc.text.replace('Voir moins', '')
        item_text = item_text.replace('[hidden information]', '')
         
        item_desc.append(item_text)
    except: 
        
        item_desc.append("No Description")
    
    print(item_title)
    print('------------------')
    time.sleep(2)

Santafe [hidden information] au compteur
------------------
2010 SEIT ابيزا
------------------
Salam j'ai une très belle mercedes B premiere main quelque retouche mais pas de peinture 50000km
------------------
Clio 4 gt line 2018 sbigha 00 farwa 00 0771531264 0541044511
------------------
Clio 4 Limited 1.5 dci
------------------
Picanto Gt Line 2019 0561155227
------------------
Blida
------------------
2013 Nissan pathphinder
------------------
تيزي وزو
------------------
[hidden information]
------------------
[hidden information]
------------------
Vendu
  · Renault Capture made in France
------------------
Gtline2013 belak touchنضيفة
------------------
Fyat hba n9iya
------------------
Vendu
------------------
Véhicule djida roller 47 miles sbigha fiha 2 bass de casse wahed afroide we lokher sebigha ctt we so
------------------
حاب اتبيع سيارة اكسيدنتي التصاور و رقم الهاتف
------------------
Tiguan trendline 2009
------------------
2015 Citroen C4
------------------
Renault clio.

# put the data in a dataframe

In [114]:
df = pd.DataFrame([item_title,item_cat,item_price,item_desc,item_loc])  
df = df.transpose() #To Transpose and make each rows as columns
df.columns=['Title', 'Category', 'Price','Description','Location'] 
 

In [115]:
df

,Title,Category,Price,Description,Location
0,Santafe [hidden information] au compteur,Véhicules,3 000 000 DA,Santafe 2014\nVoiture très propre avec un faib...,"الدار البيضاء, الجزائر"
1,2010 SEIT ابيزا,Véhicules,3 000 000 DA,الي كل من يعاني من مشاكل في غلق سيارة ابيزا وS...,"يسر, بومرداس"
2,Salam j'ai une très belle mercedes B premiere ...,Véhicules,2 800 000 DA,No Description,"الجزائر, الجزائر"
3,Clio 4 gt line 2018 sbigha 00 farwa 00 0771531...,Véhicules,2 773 456 DA,0771531264\n0541044511,"الجزائر, الجزائر"
4,Clio 4 Limited 1.5 dci,Véhicules,2 750 000 DA,Clio 4 2019 édition limited 1.5 dci machiya 22...,"بومرداس, بومرداس"
...,...,...,...,...,...
67,capteur de recul,Véhicules,1 111 111 DA,capteur de recul. Tél,"برج الكيفان, الجزائر"
68,Khawti 3ayto fa tlf,Véhicules,1 111 111 DA,"Polo 1,4 \nMotor mayaskhon mayna9ass zite \nSi...","هراوة, الجزائر"
69,Polo n9iya ta3 ksib,Véhicules,1 111 111 DA,بولو نقية تاع كسيب ماشية 192\nعطاو فيها 165 \n...,"الجزائر, الجزائر"
70,سيارة و نص و 25,Véhicules,1 111 111 DA,سبيغة : لي بارشوك برك\nموتور :5 كاري تيربو محط...,"الجزائر, الجزائر"
